# Importing libraries & dependencies

In [1]:
import random
import numpy as np

# Algorithm Parameters

In [2]:
# Define genetic algorithm parameters
pop_size = 50 # the number of individuals in each generation of the population
chrom_len = 20 #  the length of each chromosome (i.e., the number of genes) in the population.
num_generations = 50 # the total number of generations the algorithm will run for.
crossover_prob = 0.8 # the probability of a crossover (i.e., genetic recombination) occurring between two parent individuals during the reproduction step.
mutation_prob = 0.01 # the probability of a mutation occurring during the reproduction step.
num_parents = 2 # the number of parents selected from the population for reproduction in each generation.
tournament_size = 5 # he size of the tournament selection process used to select parents for reproduction. In this approach, a random subset of tournament_size individuals from the population is chosen, and the fittest individual among them is selected as a parent. This process is repeated num_parents times to select the required number of parents for reproduction.

In [3]:
# Define penalty parameters

c = 10  # constant for static penalty

alpha = 0.1  # coefficient for dynamic penalty

# Selection Mechanism

In [4]:
def selection(population, knapsack, tournament_size):
    selected = []
    for i in range(pop_size):
        tournament = random.sample(range(pop_size), tournament_size)
        fitness_values = [fitness(np.array(list(map(int, population[j]))), knapsack) for j in tournament]
        selected.append(population[tournament[np.argmax(fitness_values)]])
    return selected

- selected = []: Create an empty list to store the selected individuals.

- for i in range(pop_size): Loop over each individual in the population.

- tournament = random.sample(range(pop_size), tournament_size): Randomly select tournament_size individuals from the population to compete in a tournament. The random.sample() function is used to ensure that the same individual is not selected multiple times.

- fitness_values = [fitness(np.array(list(map(int, population[j]))), knapsack) for j in tournament]: Calculate the 
fitness value for each individual in the tournament using the fitness() function. This function takes as input a binary string (the individual), converts it to a numpy array of integers, and calculates its fitness value with respect to the knapsack problem defined in the knapsack dictionary.

- selected.append(population[tournament[np.argmax(fitness_values)]]): Select the individual with the highest fitness value from the tournament and add it to the selected list. This is done by using the np.argmax() function to find the index of the individual with the highest fitness value in the fitness_values list, and then using this index to retrieve the corresponding individual from the population list.

- return selected: Return the list of selected individuals.

Overall, the selection() function uses tournament selection to choose the individuals with the highest fitness values from a randomly selected subset of the population. The tournament_size parameter controls the number of individuals in each tournament, with larger values leading to stronger selection pressure and potentially less diversity in the population. By repeating this process for each individual in the population, the function produces a list of selected individuals that will be used as parents for producing the next generation in the genetic algorithm.






# Crossover Mechanisms

In [5]:
def one_point_crossover(parents):
    crossover_point = random.randint(1, chrom_len-1)
    offspring = parents[0][:crossover_point] + parents[1][crossover_point:]
    return offspring

- crossover_point = random.randint(1, chrom_len-1): Generate a random integer between 1 and chrom_len-1, where chrom_len is the length of the binary strings representing the individuals. This randomly selects the position at which the crossover will occur.

- offspring = parents[0][:crossover_point] + parents[1][crossover_point:]: Create the offspring by combining the first part of the binary string from the first parent up to the crossover point, with the second part of the binary string from the second parent after the crossover point. This effectively "swaps" genetic material between the parents to create a new individual.

- return offspring: Return the resulting binary string representing the offspring.

Overall, the one_point_crossover() function implements a one-point crossover operator, which is a common genetic operator used to recombine genetic material from two parents to create offspring with new genetic combinations. The function takes as input a list of two binary strings representing the parents and randomly selects a crossover point between the two strings. It then combines the genetic material from the two parents up to and after the crossover point to create a new binary string representing the offspring. The resulting offspring will have some genetic material from both parents, which can potentially lead to better solutions to the knapsack problem in the next generation of the genetic algorithm.

In [6]:
def uniform_crossover(parents):
    offspring = []
    for i in range(chrom_len):
        if random.random() < 0.5:
            offspring.append(parents[0][i])
        else:
            offspring.append(parents[1][i])
    return ''.join(offspring)

- offspring = []: Create an empty list to store the binary values for the offspring.

- for i in range(chrom_len): Loop over each bit in the binary string.

- if random.random() < 0.5:: With a probability of 0.5, randomly select one of the two parents to provide the binary value for the current bit. This effectively implements a uniform crossover operator, which has a 50% chance of selecting the binary value from each parent.

- offspring.append(parents[0][i]) or offspring.append(parents[1][i]): Depending on the outcome of the random selection in step 3, append the binary value for the current bit from either the first or second parent to the offspring list.

- return ''.join(offspring): Return the resulting binary string representing the offspring, obtained by joining the elements of the offspring list into a single string.

Overall, the uniform_crossover() function implements a uniform crossover operator, which randomly selects binary values from each parent to create a new individual. The function takes as input a list of two binary strings representing the parents, and loops over each bit in the binary strings. For each bit, it randomly selects one of the two parents to provide the binary value with a probability of 0.5. The resulting offspring will have a combination of genetic material from both parents, and this can potentially lead to better solutions to the knapsack problem in the next generation of the genetic algorithm.

In [7]:
def multi_parent_crossover(parents):
    chrom_len = len(parents[0])
    num_parents = len(parents)
    num_points = random.randint(1, chrom_len - 1)
    points = random.sample(range(1, chrom_len), num_points)
    points.sort()
    child = ''
    for i in range(num_parents):
        if i == 0:
            segment_start = 0
        else:
            segment_start = points[i - 1]
        if i == num_parents - 1:
            segment_end = chrom_len
        else:
            segment_end = points[i]
        child += parents[i][segment_start:segment_end]
    return child

- chrom_len = len(parents[0]): Get the length of the binary strings representing the individuals.

- num_parents = len(parents): Get the number of parents in the current crossover operation.

- num_points = random.randint(1, chrom_len - 1): Choose a random integer between 1 and chrom_len - 1, which represents the number of crossover points that will be used.

- points = random.sample(range(1, chrom_len), num_points): Generate num_points unique crossover points as a list of integers between 1 and chrom_len - 1.

- points.sort(): Sort the list of crossover points in ascending order.

- child = '': Create an empty string to store the binary string representing the offspring.

- for i in range(num_parents):: Loop over each parent in the list of parents.

- if i == 0: segment_start = 0: If this is the first parent, set the starting segment to 0.

- else: segment_start = points[i - 1]: Otherwise, set the starting segment to the last crossover point used in the previous parent.

- if i == num_parents - 1: segment_end = chrom_len: If this is the last parent, set the ending segment to the end of the binary string.

- else: segment_end = points[i]: Otherwise, set the ending segment to the next crossover point in the list.

- child += parents[i][segment_start:segment_end]: Append the segment of the binary string from the current parent specified by the starting and ending points to the child string.

- return child: Return the resulting binary string representing the offspring.

Overall, the multi_parent_crossover() function implements a multiple-point crossover operator, which selects multiple crossover points to recombine genetic material from multiple parents to create offspring with new genetic combinations. The function takes as input a list of binary strings representing the parents, and generates a random number of crossover points between 1 and the length of the binary strings minus 1. The crossover points are sorted in ascending order, and the binary strings are divided into segments between the crossover points. The segments are then concatenated together in the order of the parents to create the binary string representing the offspring. The resulting offspring will have genetic material from multiple parents, and this can potentially lead to better solutions to the knapsack problem in the next generation of the genetic algorithm.






# Mutation Mechanism

In [8]:
def mutation(chromosome):
    mutated_chromosome = ''
    for i in range(chrom_len):
        if random.random() < mutation_prob:
            mutated_chromosome += '1' if chromosome[i] == '0' else '0'
        else:
            mutated_chromosome += chromosome[i]
    return mutated_chromosome

- mutated_chromosome = '': Create an empty string to store the binary string representing the mutated chromosome.

- for i in range(chrom_len):: Loop over each bit in the binary string representing the chromosome.

- if random.random() < mutation_prob:: With a probability equal to the mutation rate, decide to mutate the bit.

- mutated_chromosome += '1' if chromosome[i] == '0' else '0': If the bit is to be mutated, flip the bit value (i.e., change 0 to 1 or 1 to 0) and append the result to the mutated chromosome string. If the bit is not to be mutated, append the original bit value to the mutated chromosome string.

- else: mutated_chromosome += chromosome[i]: If the bit is not to be mutated, append the original bit value to the mutated chromosome string.

- return mutated_chromosome: Return the resulting binary string representing the mutated chromosome.

Overall, the mutation() function implements a bit-flip mutation operator, which randomly flips the bit values in the binary string representing a chromosome with a certain probability. The function takes as input a binary string representing the chromosome to be mutated. For each bit in the binary string, the function decides whether to mutate the bit based on the mutation rate. If the bit is to be mutated, the function flips the bit value (i.e., changes 0 to 1 or 1 to 0) and appends the result to the mutated chromosome string. If the bit is not to be mutated, the function appends the original bit value to the mutated chromosome string. The resulting binary string represents the mutated chromosome, which may have a different combination of genetic material than the original chromosome. This introduces additional genetic diversity into the population, which can potentially lead to better solutions to the knapsack problem in the next generation of the genetic algorithm.

# Fitness Evaluation & Penalty Mechanisms

In [9]:
def fitness(chromosome, knapsack):
    
    chromosome_arr = np.array(list(map(int, chromosome)))
    total_weight = np.sum(chromosome_arr * knapsack["weights"])
    total_value = np.sum(chromosome_arr * knapsack["values"])
    
    # Dynamic Penalty
    
    if total_weight > knapsack["capacity"]:
        # Calculate penalty
        penalty = total_value - c * (total_weight - knapsack["capacity"])
        # Adjust total value based on penalty
        total_value -= penalty
        
    elif np.count_nonzero(chromosome_arr) == 0:
        # Apply static penalty if the chromosome is all zeroes
        total_value = (total_weight - knapsack["capacity"]) - c
    
    
    return total_value

The fitness function calculates the fitness value of a chromosome in the knapsack problem. It takes two arguments: chromosome, which is a binary string representing the items selected for the knapsack, and knapsack, which is a dictionary containing the capacity, weights, and values of the items in the knapsack problem.

The function first converts the binary string chromosome into a numpy array of integers using the map and list functions. It then calculates the total weight and total value of the items in the knapsack by multiplying the binary string with the weights and values arrays of the knapsack dictionary and summing the resulting arrays.

The function then applies a dynamic penalty if the total weight exceeds the capacity of the knapsack or if the chromosome is all zeroes. The penalty is calculated by multiplying the excess weight by a penalty factor c and subtracting this from the total value. The value after penalty adjustment is returned as the fitness value of the chromosome.

# GA Mechanism

In [10]:
def run_ga(pop_size, chrom_len, num_generations, crossover_prob, mutation_prob, num_parents, 
           tournament_size, crossover_method, knapsack):
  
    # Initialize list to store best solutions
    best_solutions = []

    # Run the algorithm for 30 independent runs
    for i in range(30):
        # Initialize population
        population = [''.join([str(random.randint(0, 1)) for i in range(chrom_len)]) for j in range(pop_size)]

        # Evolution loop
        for generation in range(num_generations):
            
            # Selection
            selected_population = selection(population, knapsack, tournament_size)

            # Crossover
            offspring_population = []
            
            for i in range(int(pop_size/2)):
                
                parents = random.sample(selected_population, num_parents)
                
                if random.random() < crossover_prob:
                    
                    if crossover_method == "uniform":
                        
                        offspring1 = uniform_crossover(parents)
                        offspring2 = uniform_crossover(parents[::-1])
                        
                    elif crossover_method == "one-point":
                        offspring1 = one_point_crossover(parents)
                        offspring2 = one_point_crossover(parents[::-1])
                        
                    elif crossover_method == "multi-parent":
                        offspring1 = multi_parent_crossover(parents)
                        offspring2 = multi_parent_crossover(parents[::-1])
                        
                    else:
                        raise ValueError("Invalid crossover method")
                    offspring_population += [offspring1, offspring2]
                else:
                    offspring_population += parents

            # Mutation
            mutated_population = [mutation(chromosome) for chromosome in offspring_population]

            # Evaluate fitness
            fitness_values = [fitness(np.array(list(map(int, chromosome))), knapsack) for chromosome in mutated_population]

            # Select new population
            population = [mutated_population[i] for i in np.argsort(fitness_values)[::-1][:pop_size]]

        # Record the best solution from this run
        best_solution = max(population, key=lambda chromosome: fitness(np.array(list(map(int, chromosome))), knapsack))
        best_solutions.append(best_solution)

    # Return the average of the best-of-run solutions
    avg_best_solution = sum([fitness(np.array(list(map(int, chromosome))), knapsack) for chromosome in best_solutions]) / len(best_solutions)
    return avg_best_solution


The run_ga function runs a genetic algorithm to solve the knapsack problem. The function takes several parameters, including the population size, chromosome length, number of generations, crossover and mutation probabilities, number of parents for crossover, tournament size for selection, crossover method, and the knapsack problem instance.

The function first initializes a list to store the best solutions obtained in 30 independent runs of the algorithm. It then runs the algorithm for each of these 30 independent runs.

Within each run, the function initializes a population of random binary strings, with each string representing a potential solution to the knapsack problem. It then runs an evolution loop for the specified number of generations, which includes the selection, crossover, and mutation operations.

In the selection step, the function uses tournament selection to select a subset of individuals from the population to be used as parents for the next generation. The size of the tournament is specified by the tournament_size parameter.

In the crossover step, the function randomly selects pairs of parents from the selected population and performs crossover with a probability specified by the crossover_prob parameter. The method of crossover is determined by the crossover_method parameter, which can be set to "uniform", "one-point", or "multi-parent".

If uniform crossover is selected, the function performs uniform crossover on the parents to generate two offspring. If one-point crossover is selected, the function performs one-point crossover on the parents to generate two offspring. If multi-parent crossover is selected, the function performs multi-parent crossover on the parents to generate two offspring. The offspring are added to the offspring population.

In the mutation step, the function applies mutation to the offspring population with a probability specified by the mutation_prob parameter.

After mutation, the function evaluates the fitness of the mutated population. The fitness of each individual in the population is determined by the fitness function, which takes the chromosome and knapsack problem instance as input and returns the fitness value.

The function then selects a new population from the mutated population based on the fitness values. The new population consists of the pop_size individuals with the highest fitness values.

After running the evolution loop for the specified number of generations, the function records the best solution obtained in the run, which is the individual with the highest fitness value in the final population. The function adds this best solution to the list of best solutions obtained in all runs.

Finally, the function calculates and returns the average of the best-of-run solutions obtained in the 30 independent runs.

In [11]:
# Take crossover method as input from the user

crossover_method = input("Enter the crossover method (uniform/one-point/multi-parent): ")

# First instance - no items fit in knapsack
wi = [random.randint(30, 50) for i in range(20)]
vi = [random.randint(1, 10) for i in range(20)]
W = 10
knapsack = {"weights": wi, "values": vi, "capacity": W}

print("\n\nInstance 1 - No items fit in knapsack\n\n")
best_solution = run_ga(pop_size, chrom_len, num_generations, crossover_prob, mutation_prob, num_parents, tournament_size, crossover_method, knapsack=knapsack)

# Print report
print("Weights: ", knapsack["weights"])
print("Values: ", knapsack["values"])
print("Capacity: ", knapsack["capacity"])
print("Average: ", best_solution)

# Second instance - all items fit in knapsack
wi = [random.randint(1, 10) for i in range(20)]
vi = [random.randint(30, 50) for i in range(20)]
W = 200
knapsack = {"weights": wi, "values": vi, "capacity": W}

print("\n\nInstance 2 - All items fit in knapsack\n\n")
best_solution = run_ga(pop_size, chrom_len, num_generations, crossover_prob, mutation_prob, num_parents, tournament_size, crossover_method, knapsack=knapsack)
print("Weights: ", wi)
print("Values: ", vi)
print("Capacity: ", W)
print("Average: ", best_solution)

wi = [random.randint(1, 10) for i in range(20)]
vi = [random.randint(1, 10) for i in range(20)]
W = 50
knapsack = {"weights": wi, "values": vi, "capacity": W}

print("\n\nInstance 3 - Regular case\n\n")
best_solution = run_ga(pop_size, chrom_len, num_generations, crossover_prob, mutation_prob, num_parents, tournament_size, crossover_method, knapsack=knapsack)
print("Weights: ", wi)
print("Values: ", vi)
print("Capacity: ", W)
print("Average: ", best_solution)



Enter the crossover method (uniform/one-point/multi-parent): one-point


Instance 1 - No items fit in knapsack


Weights:  [44, 35, 40, 50, 46, 46, 36, 38, 47, 50, 35, 31, 34, 40, 31, 36, 37, 46, 45, 41]
Values:  [4, 8, 6, 9, 10, 5, 4, 5, 10, 8, 6, 8, 1, 9, 8, 2, 7, 6, 8, 7]
Capacity:  10
Average:  7980.0


Instance 2 - All items fit in knapsack


Weights:  [7, 10, 10, 8, 9, 9, 4, 9, 10, 6, 3, 7, 9, 4, 3, 6, 10, 5, 9, 1]
Values:  [43, 44, 43, 36, 43, 50, 42, 31, 32, 39, 31, 36, 38, 46, 39, 48, 37, 33, 39, 30]
Capacity:  200
Average:  780.0


Instance 3 - Regular case


Weights:  [6, 2, 7, 8, 4, 8, 8, 9, 6, 9, 4, 3, 6, 4, 7, 3, 7, 6, 10, 6]
Values:  [2, 5, 7, 8, 9, 6, 3, 3, 10, 2, 4, 10, 7, 8, 3, 3, 1, 1, 3, 1]
Capacity:  50
Average:  730.0


This code is running three different instances of the knapsack problem with different sets of weights, values, and capacity. For each instance, it calls the run_ga function with the specified parameters including the crossover method selected by the user, and the knapsack instance.

After running the genetic algorithm on each instance, it prints the weights, values, and capacity of the knapsack, and the average best solution obtained over 30 independent runs of the algorithm.

The first instance represents a case where no items fit in the knapsack. The second instance represents a case where all items fit in the knapsack. The third instance represents a regular case with randomly generated weights, values, and capacity.